In [1]:
!pip install fake_useragent

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 5.0 MB/s eta 0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from fake_useragent import UserAgent

In [20]:
keyword = "AI"
location = "India"

In [21]:
search_query = keyword.replace(" ", "%20")
search_url = f"https://www.linkedin.com/jobs/search/?keywords={search_query}&location={location}"
print("Scraping from:", search_url)

Scraping from: https://www.linkedin.com/jobs/search/?keywords=AI&location=India


In [22]:
ua = UserAgent()
headers = {
    'User-Agent': ua.random,
    'Accept-Language': 'en-US,en;q=0.9'
}

response = requests.get(search_url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    jobs = soup.find_all('div', class_='base-card')[:10]  # Top 10 jobs only
    print(f"✅ Found {len(jobs)} jobs.")
else:
    print("❌ Failed to fetch page. Status:", response.status_code)

✅ Found 7 jobs.


In [23]:
job_list = []

for job in jobs:
    try:
        title = job.find('h3').text.strip()
        company = job.find('h4').text.strip()
        location = job.find('span', class_='job-search-card__location').text.strip()
        job_url = job.find('a')['href'].strip()

        job_list.append({
            'Job Title': title,
            'Company': company,
            'Location': location,
            'Job URL': job_url,
        })
    except Exception as e:
        print("Error:", e)

df_jobs = pd.DataFrame(job_list)
df_jobs.head()

,Job Title,Company,Location,Job URL
0,AI/ML,Infosys,"Bengaluru East, Karnataka, India",https://in.linkedin.com/jobs/view/ai-ml-at-inf...
1,AI/ML,Infosys,"Bengaluru East, Karnataka, India",https://in.linkedin.com/jobs/view/ai-ml-at-inf...
2,Data Scientist,PepsiCo,"Hyderabad, Telangana, India",https://in.linkedin.com/jobs/view/data-scienti...
3,**/** ******** ** [****-*****],********'*,"*********, *********, *****",https://in.linkedin.com/jobs/view/ai-ml-engine...
4,**/** *********,*** ************ *******,"*********, *********, *****",https://in.linkedin.com/jobs/view/ai-ml-develo...


In [24]:
HUNTER_API_KEY = "9d2a5449b8c244cdb97060c5a4356f0d6799c26c"

def enrich_with_email(company_name):
    try:
        url = f"https://api.hunter.io/v2/domain-search?company={company_name}&api_key={HUNTER_API_KEY}"
        r = requests.get(url)
        result = r.json()

        if result.get("data") and result["data"].get("emails"):
            return result["data"]["emails"][0]["value"]
        else:
            return "Not Found"
    except:
        return "Error"

In [25]:
emails = []
for idx, row in df_jobs.iterrows():
    email = enrich_with_email(row['Company'])
    print(f"{row['Company']} ➤ {email}")
    emails.append(email)
    time.sleep(1)  # Prevent API rate-limit

df_jobs["Email"] = emails
df_jobs.head()

Infosys ➤ harsh_saxena@infosys.com
Infosys ➤ harsh_saxena@infosys.com
PepsiCo ➤ meenakshi.mishra@pepsico.com
********'* ➤ Not Found
*** ************ ******* ➤ Not Found
********'* ➤ Not Found
******** ******* ➤ Not Found


,Job Title,Company,Location,Job URL,Email
0,AI/ML,Infosys,"Bengaluru East, Karnataka, India",https://in.linkedin.com/jobs/view/ai-ml-at-inf...,harsh_saxena@infosys.com
1,AI/ML,Infosys,"Bengaluru East, Karnataka, India",https://in.linkedin.com/jobs/view/ai-ml-at-inf...,harsh_saxena@infosys.com
2,Data Scientist,PepsiCo,"Hyderabad, Telangana, India",https://in.linkedin.com/jobs/view/data-scienti...,meenakshi.mishra@pepsico.com
3,**/** ******** ** [****-*****],********'*,"*********, *********, *****",https://in.linkedin.com/jobs/view/ai-ml-engine...,Not Found
4,**/** *********,*** ************ *******,"*********, *********, *****",https://in.linkedin.com/jobs/view/ai-ml-develo...,Not Found


In [26]:
df_jobs.to_csv("lead_list.csv", index=False)
df_jobs

,Job Title,Company,Location,Job URL,Email
0,AI/ML,Infosys,"Bengaluru East, Karnataka, India",https://in.linkedin.com/jobs/view/ai-ml-at-inf...,harsh_saxena@infosys.com
1,AI/ML,Infosys,"Bengaluru East, Karnataka, India",https://in.linkedin.com/jobs/view/ai-ml-at-inf...,harsh_saxena@infosys.com
2,Data Scientist,PepsiCo,"Hyderabad, Telangana, India",https://in.linkedin.com/jobs/view/data-scienti...,meenakshi.mishra@pepsico.com
3,**/** ******** ** [****-*****],********'*,"*********, *********, *****",https://in.linkedin.com/jobs/view/ai-ml-engine...,Not Found
4,**/** *********,*** ************ *******,"*********, *********, *****",https://in.linkedin.com/jobs/view/ai-ml-develo...,Not Found
5,"*******, **** ************* & ******** *******...",********'*,"*********, *********, *****",https://in.linkedin.com/jobs/view/manager-data...,Not Found
6,"**** ********* (***, ******, ****)",******** *******,"********, *******, *****",https://in.linkedin.com/jobs/view/data-analyti...,Not Found


In [27]:
from google.colab import files
files.download("lead_list.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
from IPython.display import display, HTML
display(HTML(df_jobs.to_html(render_links=True)))

,Job Title,Company,Location,Job URL,Email
0,AI/ML,Infosys,"Bengaluru East, Karnataka, India",https://in.linkedin.com/jobs/view/ai-ml-at-infosys-4218976284?position=1&pageNum=0&refId=qYdxgat0me5kWjcZN%2F9z%2BA%3D%3D&trackingId=juyyGKQQHQIrY2Z0h64L5g%3D%3D,harsh_saxena@infosys.com
1,AI/ML,Infosys,"Bengaluru East, Karnataka, India",https://in.linkedin.com/jobs/view/ai-ml-at-infosys-4218974529?position=2&pageNum=0&refId=qYdxgat0me5kWjcZN%2F9z%2BA%3D%3D&trackingId=boz9Cfs%2BC6MD3L8k1dgedA%3D%3D,harsh_saxena@infosys.com
2,Data Scientist,PepsiCo,"Hyderabad, Telangana, India",https://in.linkedin.com/jobs/view/data-scientist-at-pepsico-4238229591?position=3&pageNum=0&refId=qYdxgat0me5kWjcZN%2F9z%2BA%3D%3D&trackingId=GMPnQYN0WlXqyMl8Re9EyA%3D%3D,meenakshi.mishra@pepsico.com
3,**/** ******** ** [****-*****],********'*,"*********, *********, *****",https://in.linkedin.com/jobs/view/ai-ml-engineer-ii-t500-17910-at-mcdonald-s-4228935169?position=4&pageNum=0&refId=qYdxgat0me5kWjcZN%2F9z%2BA%3D%3D&trackingId=DyEzOTkf4BxuTR9hmZ55nQ%3D%3D,Not Found
4,**/** *********,*** ************ *******,"*********, *********, *****",https://in.linkedin.com/jobs/view/ai-ml-developer-at-pwc-acceleration-centers-4238469565?position=5&pageNum=0&refId=qYdxgat0me5kWjcZN%2F9z%2BA%3D%3D&trackingId=nT9XfeL01QiTLCMzRfYjrg%3D%3D,Not Found
5,"*******, **** ************* & ******** ************ [****-*****]",********'*,"*********, *********, *****",https://in.linkedin.com/jobs/view/manager-data-visualization-business-intelligence-t500-18181-at-mcdonald-s-4237598782?position=6&pageNum=0&refId=qYdxgat0me5kWjcZN%2F9z%2BA%3D%3D&trackingId=6o1QxBNxnqw6hXk6gPuvyw%3D%3D,Not Found
6,"**** ********* (***, ******, ****)",******** *******,"********, *******, *****",https://in.linkedin.com/jobs/view/data-analytics-sql-python-hive-at-american-express-4240349000?position=7&pageNum=0&refId=qYdxgat0me5kWjcZN%2F9z%2BA%3D%3D&trackingId=KJXpAv%2BKnnM%2ByENmXvOnhQ%3D%3D,Not Found
